# Importing Libraries

In [1]:
#Libraries
!pip install langdetect
!pip install google_trans_new
!pip3 install googletrans==3.1.0a0
import pandas as pd
from langdetect import detect
from google.colab import drive
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import operator
import string
import nltk

from collections import Counter

from google_trans_new import google_translator

from googletrans import Translator

from textblob import TextBlob


#Formatting
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 4.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=adac3c5b48c139cb30988ed9b8d54be0b03fad5c4b7c6b4ba69528c965345752
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 1.3 MB 8.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-non

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Reading the File

In [194]:
drive.mount("/google_drive")
WORKSPACE_PATH = "/google_drive/My Drive/MBD Capstone"


xls = pd.ExcelFile(WORKSPACE_PATH+ '/ie_input_data_no_character.xlsx')
df = pd.read_excel(xls, 'Sheet1')

Drive already mounted at /google_drive; to attempt to forcibly remount, call drive.mount("/google_drive", force_remount=True).


# Translating `text_answers`

In [195]:
traductor = Translator()
texto = df['text_answers'].str.lower()
texto = texto.fillna(' ')
texto_trad = pd.Series([])

In [196]:
print(traductor.translate('si', dest='en'))

Translated(src=ro, dest=en, text=and, pronunciation=and, extra_data="{'translat...")


In [197]:
for cont in range(len(texto)):
  traduccion = traductor.translate(texto[cont], dest='en')
  if traduccion.src == 'ro':
    traduccion = traductor.translate(texto[cont], src='es', dest='en')
    texto_trad[cont] = traduccion.text
  else:
    texto_trad[cont] = traduccion.text
#print(texto_trad)

In [198]:
len(df.columns)

7

In [199]:
df.insert(7, 'texto_traducido', texto_trad)

In [200]:
df.columns

Index(['Unnamed: 0', 'unique_id', 'text', 'idioma', 'show_up', 'text_answers',
       'last_answer', 'texto_traducido'],
      dtype='object')

In [201]:
df.head()

,Unnamed: 0,unique_id,text,idioma,show_up,text_answers,last_answer,texto_traducido
0,0,1824583,"Hola Jorge. Mi nombre es Julia Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA el jueves a las 19:00 horas. ¿Confirma su asistencia? Si , gracias. Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,1,"Si , gracias.","Si , gracias.","yes, thanks."
1,1,1828620,"Hola Rosario. Mi nombre es Julia Tiene cita de Ecografia en Centro de Diagnóstico Natalio Rivas el viernes a las 12:40 horas. ¿Confirma su asistencia? Si Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,1,Si,Si,Yes
2,2,1829301,"Hola Ana. Mi nombre es Julia Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA el sábado a las 12:30 horas. ¿Confirma su asistencia? Si allí estaré!!!, Gracias Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,0,"Si allí estaré!!!, Gracias","Si allí estaré!!!, Gracias","yes i will be there!!!, thanks"
3,3,1831509,"Hola, buenos días Jose, Mi nombre es Julia. Espero que esté bien. Le escribo para confirmar su cita de Ecografia el viernes a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. ¿Confirma la cita? Me mantengo a la espera de su respuesta. Saludos, Julia Eniax Atencion al Paciente Buenos días Jose, Le escribo de nuevo para solicitar la confirmación de la cita que tiene usted de Ecografia el viernes a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. Esperamos su respuesta para proceder a la confirmación de su cita. ¿Podrá asistir a la cita? Saludos, Julia Eniax Atencion al Paciente Hola Jose. Espero que esté bien Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA mañana a las 18:15 horas. ¿Confirma su asistencia? Si Estimado Jose, Recuerde su cita de Ecografia para hoy a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. Le rogamos acuda a su prueba con 10 minutos de antelación para gestionar su admisión. En caso de no poder asistir, por favor, le pedimos informar por este medio. Agradecemos su colaboración debido a la alta demanda de citas. Saludos, Julia Eniax Atencion al Paciente",es,1,Si,Si,Yes
4,4,1833572,"Hola Gloria. Espero que esté bien Tiene cita de Resonancia Magnética en Centro de Diagnóstico Granada - Clínica CEDISA hoy a las 15:00 horas. ¿Confirma su asistencia? ,si",es,1,",si",",si",",Yes"


# Creating `polarity` and `sentiment` columns

In [202]:
polaridad = df['texto_traducido'].apply(lambda comment_eng: TextBlob(comment_eng).polarity)
df.insert(8, 'polaridad', polaridad)

sentimiento = ['positivo' if v >= 0 else 'negativo' for v in df['polaridad']]
df.insert(9, 'sentimiento', sentimiento)

In [203]:
df['text_answers'] = df['text_answers'].fillna("")

# Creating `senimiento_def` with the answers that contains **"cancel...", "no" and "repro..."*

In [204]:
df['sentimiento_def'] = df['text_answers'].str.lower().str.contains(r'\bcancel?|\bno\b|\brepro?', na=False).replace(True, 'negativo').replace(False, 'positivo')

In [205]:
df = df.drop(columns=['Unnamed: 0', 'sentimiento'])

In [206]:
df.head()

,unique_id,text,idioma,show_up,text_answers,last_answer,texto_traducido,polaridad,sentimiento_def
0,1824583,"Hola Jorge. Mi nombre es Julia Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA el jueves a las 19:00 horas. ¿Confirma su asistencia? Si , gracias. Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,1,"Si , gracias.","Si , gracias.","yes, thanks.",0.2,positivo
1,1828620,"Hola Rosario. Mi nombre es Julia Tiene cita de Ecografia en Centro de Diagnóstico Natalio Rivas el viernes a las 12:40 horas. ¿Confirma su asistencia? Si Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,1,Si,Si,Yes,0.0,positivo
2,1829301,"Hola Ana. Mi nombre es Julia Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA el sábado a las 12:30 horas. ¿Confirma su asistencia? Si allí estaré!!!, Gracias Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,0,"Si allí estaré!!!, Gracias","Si allí estaré!!!, Gracias","yes i will be there!!!, thanks",0.2,positivo
3,1831509,"Hola, buenos días Jose, Mi nombre es Julia. Espero que esté bien. Le escribo para confirmar su cita de Ecografia el viernes a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. ¿Confirma la cita? Me mantengo a la espera de su respuesta. Saludos, Julia Eniax Atencion al Paciente Buenos días Jose, Le escribo de nuevo para solicitar la confirmación de la cita que tiene usted de Ecografia el viernes a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. Esperamos su respuesta para proceder a la confirmación de su cita. ¿Podrá asistir a la cita? Saludos, Julia Eniax Atencion al Paciente Hola Jose. Espero que esté bien Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA mañana a las 18:15 horas. ¿Confirma su asistencia? Si Estimado Jose, Recuerde su cita de Ecografia para hoy a las 18:15 horas en Hospital INMACULADA Grupo HLA, ubicado en C/ Alejandro Otero, 8, 18004 Granada. Le rogamos acuda a su prueba con 10 minutos de antelación para gestionar su admisión. En caso de no poder asistir, por favor, le pedimos informar por este medio. Agradecemos su colaboración debido a la alta demanda de citas. Saludos, Julia Eniax Atencion al Paciente",es,1,Si,Si,Yes,0.0,positivo
4,1833572,"Hola Gloria. Espero que esté bien Tiene cita de Resonancia Magnética en Centro de Diagnóstico Granada - Clínica CEDISA hoy a las 15:00 horas. ¿Confirma su asistencia? ,si",es,1,",si",",si",",Yes",0.0,positivo


In [207]:
df.loc[df['sentimiento_def'] == 'positivo']

Output hidden; open in https://colab.research.google.com to view.

In [208]:
df.columns

Index(['unique_id', 'text', 'idioma', 'show_up', 'text_answers', 'last_answer',
       'texto_traducido', 'polaridad', 'sentimiento_def'],
      dtype='object')

In [209]:
xls_1 = pd.ExcelFile(WORKSPACE_PATH+ '/JLB/numeric_vals.xlsx')
df_1 = pd.read_excel(xls_1, 'Sheet1')

In [210]:
df_1.head(1)

,Unnamed: 0,unique_id,text,idioma,show_up,text_answers,last_answer,is_assistant|mean,is_assistant|sum,is_assistant|median,is_patient|mean,is_patient|sum,is_patient|median,word_count|mean,word_count|sum,word_count|median,patient_word_count|mean,patient_word_count|sum,patient_word_count|median,assistant_word_count|mean,assistant_word_count|sum,assistant_word_count|median,unique_word_count|mean,unique_word_count|sum,unique_word_count|median,patient_unique_word_count|mean,patient_unique_word_count|sum,patient_unique_word_count|median,assistant_unique_word_count|mean,assistant_unique_word_count|sum,assistant_unique_word_count|median,punctuation_count|mean,punctuation_count|sum,punctuation_count|median,patient_punctuation_count|mean,patient_punctuation_count|sum,patient_punctuation_count|median,assistant_punctuation_count|mean,assistant_punctuation_count|sum,assistant_punctuation_count|median
0,0,1824583,"Hola Jorge. Mi nombre es Julia Tiene cita de Ecografia en Hospital INMACULADA Grupo HLA el jueves a las 19:00 horas. ¿Confirma su asistencia? Si , gracias. Hola Espero que esté bien Le recuerdo que tiene una cita con Si no puede asistir, rogamos nos avise para poder asignar la cita a otro paciente.",es,1,"Si , gracias.","Si , gracias.",0.666667,2,1.0,0.333333,1,0.0,18.0,54,24.0,1.0,3,0.0,17.0,51,24.0,17.333333,52,24.0,1.0,3,0.0,16.333333,49,24.0,2.666667,8,2.0,0.666667,2,0.0,2.0,6,2.0


In [211]:
df_def = pd.merge(df_1, df[['texto_traducido', 'polaridad', 'sentimiento_def']], left_index=True, right_index=True)

In [212]:
xls_analysis_words = pd.ExcelFile(WORKSPACE_PATH+ '/df_analysis_word.xlsx')
df_analysis_words = pd.read_excel(xls_analysis_words, 'Sheet1')

In [213]:
df_analysis_def = pd.merge(df[['text','text_answers', 'texto_traducido', 'polaridad', 'sentimiento_def']], df_analysis_words, left_index=True, right_index=True)

In [214]:
len(df)

790

# Change accent

In [215]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

print(normalize("¡Hólá, múndó!"))
print(normalize("¡HÓLÁ, MÚNDÓ!"))

¡Hola, mundo!
¡HOLA, MUNDO!


In [216]:
df_def = df_def.drop(columns='Unnamed: 0')

In [217]:
normalize(df_def['text_answers'][2])

'Si alli estare!!!, Gracias'

In [218]:
type(df_def['text_answers'][0])

str

In [219]:
df_def['text_answers'] = df_def['text_answers'].astype('str')

In [220]:
df_def['text_answers_lower_case_no_accent'] = df_def['text_answers'].str.lower().apply(lambda x: normalize(x))

In [221]:
df_def[['text_answers', 'text_answers_lower_case_no_accent']].head(4)

,text_answers,text_answers_lower_case_no_accent
0,"Si , gracias.","si , gracias."
1,Si,si
2,"Si allí estaré!!!, Gracias","si alli estare!!!, gracias"
3,Si,si


In [222]:
df_def['last_answer'] = df_def['last_answer'].fillna(" ").astype(str)

In [223]:
df_def['last_answers_lower_case_no_accent'] = df_def['last_answer'].str.lower().apply(lambda x: normalize(x))

In [224]:
df_def[['last_answer', 'last_answers_lower_case_no_accent']].head(10)

,last_answer,last_answers_lower_case_no_accent
0,"Si , gracias.","si , gracias."
1,Si,si
2,"Si allí estaré!!!, Gracias","si alli estare!!!, gracias"
3,Si,si
4,",si",",si"
5,Confirmo,confirmo
6,Ok confirmada,ok confirmada
7,Si,si
8,En la sala de espera,en la sala de espera
9,"Hola, buenas yo no tengo cita hoy y no se que empresa son.","hola, buenas yo no tengo cita hoy y no se que empresa son."


In [225]:
df_def['last_answers_lower_case_no_accent'].str.strip().str.replace(r'[^\w\s]+', '').str.replace(r'  +', ' ').str.split(" ", expand = True).replace([None], "").head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138
0,si,gracias,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,si,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,si,alli,estare,gracias,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,si,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,si,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,confirmo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,ok,confirmada,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,si,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,en,la,sala,de,espera,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,hola,buenas,yo,no,tengo,cita,hoy,y,no,se,que,empresa,son,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [226]:
word_analysis_df = df_def['last_answers_lower_case_no_accent'].str.strip().str.replace(r'[^\w\s]+', '').str.replace(r'  +', ' ').str.split(" ", expand = True).replace([None], "")

In [231]:
df_def = df_def[['unique_id', 'text', 'idioma', 'show_up', 'text_answers', 'last_answer',
       'is_assistant|mean', 'is_assistant|sum', 'is_assistant|median',
       'is_patient|mean', 'is_patient|sum', 'is_patient|median',
       'word_count|mean', 'word_count|sum', 'word_count|median',
       'patient_word_count|mean', 'patient_word_count|sum',
       'patient_word_count|median', 'assistant_word_count|mean',
       'assistant_word_count|sum', 'assistant_word_count|median',
       'unique_word_count|mean', 'unique_word_count|sum',
       'unique_word_count|median', 'patient_unique_word_count|mean',
       'patient_unique_word_count|sum', 'patient_unique_word_count|median',
       'assistant_unique_word_count|mean', 'assistant_unique_word_count|sum',
       'assistant_unique_word_count|median', 'punctuation_count|mean',
       'punctuation_count|sum', 'punctuation_count|median',
       'patient_punctuation_count|mean', 'patient_punctuation_count|sum',
       'patient_punctuation_count|median', 'assistant_punctuation_count|mean',
       'assistant_punctuation_count|sum', 'assistant_punctuation_count|median',
       'texto_traducido', 'polaridad', 'sentimiento_def',
       'text_answers_lower_case_no_accent',
       'last_answers_lower_case_no_accent']]

In [228]:
#df_def = pd.merge(df_def, word_analysis_df, left_index=True, right_index=True)

# Write to Excel

In [232]:
df_def.to_excel(WORKSPACE_PATH+ '/JLB/sentiment_analysis.xlsx')

In [233]:
word_analysis_df.to_excel(WORKSPACE_PATH+ '/JLB/df_analysis_words_sentiment_analysis.xlsx')